In [1]:
from google.colab import drive 
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Chatbot'

Mounted at /content/drive


In [2]:
import json
import string
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
data_file = open(data_root + "/intents.json").read()
data = json.loads(data_file)


In [6]:
words = []
classes = []
dataX = []
dataY = []
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    dataX.append(pattern)
    dataY.append(intent["tag"])
  if intent["tag"] not in classes:
    classes.append(intent['tag'])
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))


In [7]:
training = []
out_empty = [0]* len(classes)
for idx, doc in enumerate(dataX):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(dataY[idx])] = 1
    training.append([bow,output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
trainX= np.array(list(training[:,0]))
trainY=  np.array(list(training[:,1]))

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(trainX[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(trainY[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss = 'categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=trainX,y=trainY,epochs=150,verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               15360     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 28)                1820      
                                                                 
Total params: 25,436
Trainable params: 25,436
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
380/380 [==========================

In [9]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text,vocab):
  tokens = clean_text(text)
  bow = [0]*len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word == w:
        bow[idx]= 1
  return np.array(bow)

def pred_class(text,vocab,labels):
  bow = bag_of_words(text,vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[indx,res]for indx,res in enumerate(result) if res> thresh]
  y_pred.sort(key = lambda x: x[1],reverse = True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list,intents_json):
  if len(intents_list) == 0:
    result = "Sorry no bueno"
  else: 
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [10]:
print("press 0")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message,words,classes)
  result =  get_response(intents,data)
  print(result)

press 0
hi
1/1 [==============================] - 0s 83ms/step
What u need boo?
im not good
1/1 [==============================] - 0s 17ms/step
Cool
can you help me
1/1 [==============================] - 0s 18ms/step
I am a general purpose chatbot. My capabilities are : 
 1. I can chat with you. Try asking me for jokes or riddles! 
 2. Ask me the date and time 
 3. I can google search for you. Use format google: your query 
 4. I can get the present weather for any city. Use format weather: city name 
 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' 
 6. I can get you the top 10 trending songs globally. Type 'songs' 
 7. I can set a timer for you. Enter 'set a timer: minutes to timer' 
 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' 
 For suggestions to help me improve, send an email to ted.thedlbot.suggestions@gmail.com . Thank you!! 
0
